In [44]:
from io import StringIO
from dotenv import load_dotenv

import os
import boto3
import pandas as pd


In [45]:
aws_access_key_id = os.getenv("AWS_ACCESS_KEY")
aws_secret_key = os.getenv("AWS_SECRET_KEY")

In [46]:
load_dotenv()

True

In [47]:
def read_csv_from_s3(bucket: str, path: str, filename: str) -> pd.DataFrame:
    """Downloads a csv file from an s3 bucket.
    
    Parameters
    ----------
    bucket : str
        The bucket where the files at.
    path : str
        The folders to the file.
    filename : str
        Name of the file.
        
    Returns
    -------
    pd.DataFrame
        A DataFrame of the Downloaded file.
    
    """
    s3 = boto3.client("s3", aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_key)
    
    full_path = f"{path}{filename}"
    
    object = s3.get_object(Bucket = bucket, Key = full_path)
    object = object["Body"].read().decode("utf-8")
    output_df = pd.read_csv(StringIO(object))
    
    return output_df

In [48]:
s3 = boto3.client("s3")
bucket = "cubix-chicago-taxi-vti"

payment_type_path = "transformed_data/payment_type/"

payment_type = read_csv_from_s3(bucket = bucket, path = payment_type_path, filename = "payment_type_master.csv")

In [50]:
payment_type

,payment_type_id,payment_type
0,1,Mobile
1,2,Credit Card
2,3,Cash
3,4,Prcard
4,5,Unknown
5,6,Dispute
6,7,No Charge
